In [ ]:
%pip install scipy

In [2]:
import os
os.chdir('..')

In [3]:
from dataclasses import dataclass
from pathlib import Path
from box import ConfigBox

@dataclass(frozen=True)
class ModelTrainingConfig:
    artifact_dir:Path
    trained_model_filepath:Path
    updated_base_model_path:Path
    training_data_dir:Path
    params:ConfigBox
    

In [4]:
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    artifact_dir:Path
    tensorboard_log_dir:Path
    checkpoint_model_filepath:Path

from pathlib import Path
from DeepClassifier.constants import *
from DeepClassifier.utils import read_yaml_file, make_directories


class ConfigurationManager:

    def __init__(
            self,
            config_filepath:Path=CONFIG_FILEPATH,
            params_filepath:Path=PARAMS_FILEPATH,
        ):
        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)


    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        prepare_callbacks_config_info = self.config.prepare_callbacks

        make_directories([
                Path(prepare_callbacks_config_info.artifact_dir),
                Path(prepare_callbacks_config_info.tensorboard_log_dir),
                Path(os.path.dirname(prepare_callbacks_config_info.checkpoint_model_filepath))
            ])

        prepare_callbacks_config = PrepareCallbacksConfig(
                                        artifact_dir=Path(prepare_callbacks_config_info.artifact_dir),
                                        tensorboard_log_dir=Path(prepare_callbacks_config_info.tensorboard_log_dir),
                                        checkpoint_model_filepath=Path(prepare_callbacks_config_info.checkpoint_model_filepath)
                                    )

        return prepare_callbacks_config
        

    def get_model_training_config(self) -> ModelTrainingConfig:
        model_training_config_info = self.config.model_training
        
        make_directories([
            Path(model_training_config_info.artifact_dir),
        ])
        
        training_data_dir = os.path.join(
            self.config.data_ingestion.unzipped_data_dir,
            "PetImages"
        )
        model_training_config = ModelTrainingConfig(
                                    artifact_dir=Path(model_training_config_info.artifact_dir),
                                    trained_model_filepath=Path(model_training_config_info.trained_model_filepath),
                                    updated_base_model_path=Path(self.config.prepare_base_model.updated_base_model_path),
                                    params=self.params,
                                    training_data_dir=Path(training_data_dir)
                                )

        return model_training_config

In [5]:
import tensorflow as tf


class ModelTraining:
    def __init__(
            self,
            model_training_config:ModelTrainingConfig,

        ):
        self.model_training_config = model_training_config


    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.model_training_config.updated_base_model_path
        )

    def get_train_valid_generator(self):
        data_generator_kwargs = dict(
            rescale=1/255.,
            validation_split=0.20,
        )

        valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        data_flow_kwargs = dict(
            target_size=self.model_training_config.params.IMAGE_SHAPE[:-1],
            batch_size=self.model_training_config.params.BATCH_SIZE,
            interpolation='bilinear'
        )


        self.valid_data_generator =valid_generator.flow_from_directory(
            self.model_training_config.training_data_dir,
            subset='validation',
            shuffle=False,
            **data_flow_kwargs
        )

        if self.model_training_config.params.AUGMENTATION:
            train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **data_generator_kwargs
            )
        else:
            train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                **data_generator_kwargs
            )
        
        self.train_data_generator = train_generator.flow_from_directory(
            self.model_training_config.training_data_dir,
            subset='training',
            shuffle=True,
            **data_flow_kwargs
        )


    @staticmethod
    def save_model(path:Path, model:tf.keras.models.Model) -> None:
        model.save(path)


    def train(self, callbacks:list):
        self.steps_per_epoch=self.train_data_generator.samples // self.train_data_generator.batch_size
        self.validation_steps=self.valid_data_generator.samples // self.valid_data_generator.batch_size

        self.model.fit(
            self.train_data_generator,
            epochs=self.model_training_config.params.EPOCHS,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_data_generator,
            callbacks=callbacks
        )

        self.save_model(
            path=self.model_training_config.trained_model_filepath,
            model=self.model
        )
        

In [7]:
try:
    config = ConfigurationManager()

    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(prepare_callbacks_config)
    callbacks = prepare_callbacks.get_callbacks()

    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(
        model_training_config,
    )

    model_training.get_base_model()
    model_training.get_train_valid_generator()
    model_training.train(
        callbacks=callbacks
    )
except Exception as e:
    raise e

Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
 945/1250 [=====================>........] - ETA: 9:08 - loss: 7.0506 - accuracy: 0.6658

d:\Full Stack Data Science\Python Project\DL\Current Batch\Deep_CNN_Classifier\env\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1250/1250 [==============================] - 2801s 2s/step - loss: 6.2914 - accuracy: 0.6883 - val_loss: 4.9902 - val_accuracy: 0.7244


In [6]:
# from DeepClassifier.entity import PrepareCallbacksConfig
import tensorflow as tf
import time

class PrepareCallbacks:
    def __init__(self, config:PrepareCallbacksConfig) -> None:
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        log_dir = os.path.join(
            self.config.tensorboard_log_dir, 
            f"tb_logs_at_timestamp"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=log_dir)


    @property
    def _create_ckpt_callbacks(self):
        # timestamp = time.strftime("")
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True,
            mode='auto'
        )

    @property
    def _create_early_stopping_callbacks(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='auto',
            patience=10,
            restore_best_weights=True
        )

    def get_callbacks(self) -> list:
        callbacks = [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

        return callbacks